In [2]:
import argparse
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sched_heuristic as heu
import pickle
from scipy.stats import kendalltau as tau, spearmanr as rho
from collections import defaultdict
from sched_solver import random_shuffle
import util as util_
from concurrent.futures import ProcessPoolExecutor

In [3]:
num_proc = 2
use_deadline = True
num_tasks = 16

In [4]:
!ls | grep torchmodel

In [5]:
ret = []
beg = "10"
end = "190"
start=False
for x in [str(x) for x in range(10, num_proc * 100, 10)]:
    if beg == x:
        start = True
    if start:
        if len(x) == 2:
            ret.append('0.' + x)
        else:
            ret.append(x[:len(x) - 2] + '.' + x[len(x) - 2:])
    if end == x:
        start = False

In [6]:
util_range = ret

In [11]:
ls models

'p2-t16-d0-l[0.10, 0.90].torchmodel'  'p4-t16-d0-l[3.00, 3.90].torchmodel'
'p2-t16-d0-l[1.00, 1.90].torchmodel'  'p4-t32-d0-l[0.10, 0.90].torchmodel'
'p4-t16-d0-l[0.10, 0.90].torchmodel'  'p4-t32-d0-l[1.00, 1.90].torchmodel'
'p4-t16-d0-l[1.00, 1.90].torchmodel'  'p4-t32-d0-l[2.00, 2.90].torchmodel'
'p4-t16-d0-l[2.00, 2.90].torchmodel'


In [12]:
model = torch.load("models/p2-t16-d0-l[0.10, 0.90].torchmodel")

In [13]:
executor = executor = ProcessPoolExecutor(max_workers=4)

In [14]:
def wrap(input):
    res_map = defaultdict(lambda: list())
    idx, num_tasks, num_procs, util, model  = input 
    with open("eval/%d-%d/%s" % (num_proc, num_tasks, util), 'rb') as f:
        train_dataset = pickle.load(f)
    i = 0
    for x, y in train_dataset:
        if i == 10000:
            break
        i += 1
        if i % 10 == 0:
            print("idx",idx, "iter", i)
        r, p, order = model.forward_np(torch.from_numpy(y).unsqueeze(0).repeat(k, 1, 1), argmax=False)
        order_np = order.detach().cpu().numpy()
        new_array = [tuple(row) for row in order_np]
        new_order = np.unique(new_array, axis=0)
        true = False
        for order in new_order:
            priority = np.zeros_like(order)
            for _i in range(len(order)):
                priority[order[_i]] = num_tasks - 1 - _i
            ret = heu.test_RTA_LC(y, num_proc, priority, True)
            if ret:
                true = True
                break
        res_map[util].append(true)
    return dict(res_map)

In [15]:
utils = ["0.50", "0.60", "0.70", "0.80", "0.90"]
ret = []
for x in executor.map(wrap, [(i, num_tasks, num_proc, util, model) for i, util in enumerate(utils)]):
    ret.append(x)
a = {}
for x in ret:
    a.update(x)
a

Process ForkProcess-1:
Process ForkProcess-2:
Traceback (most recent call last):
  File "/home/ita/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ita/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process ForkProcess-3:
  File "/home/ita/anaconda3/lib/python3.7/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ita/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkProcess-4:
  File "/home/ita/anaconda3/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/home/ita/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ita/anaconda3/lib/python3.7/site-packages/torch/multiprocessing/red

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [11]:
with open("4-32-dl-0.50-0.90.pkl", 'wb') as f:
    pickle.dump(a, f)

In [12]:
with open("2-8-dl-0.50-0.90.pkl", 'rb') as f:
    a = pickle.load(f)
for k ,i in a.items():
    print(np.sum(i))

EOFError: Ran out of input